In [1]:
from lxml import html, etree
import requests
import pandas as pd
import regex
import os

# parsing by xpath

In [2]:
s = requests.Session() 
s.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0'
    })

In [3]:
def get_source_code(url, session):
    response = session.get(url)
    
    return response.text

In [4]:
def save_text(text, filename):
    with open(filename, 'w') as output:
        output.write(text)

In [5]:
def read_text(filename):
    with open(filename, 'r') as input_:
        text = input_.read()

    return text

In [55]:
url_test = 'https://www.marketwatch.com/search?q=microsoft&m=Keyword&rpp=100&mp=159&bd=false&rs=true'
news_list_test = []
extract_headlines(url_test, news_list_test)

100

In [7]:
def extract_headlines(url, session, news_list):    
    text = get_source_code(url, session)
    tree = html.fromstring(text)
    headlines = tree.xpath('.//div[@class = "searchresult"]/a/text()')
    dates = tree.xpath('.//div[@class="deemphasized"]/span/text()')
    links = tree.xpath('.//div[@class = "searchresult"]/a/@href')
    
    for headline, date, link in zip(headlines, dates, links):
        news_list.append({
            'date' : date,
            'headline' : headline,
            'link' : link
        })
        
    return len(headlines)

In [14]:
def load_news(url_first_page, session):
    news_list = []
    extract_headlines(url=url_first_page, session=session, news_list=news_list)
    print('Loaded headlines: ', len(news_list), flush=True)
    
    num = 1
    news_quantity = 100
    while news_quantity < 1000:
        current_page_url = url_first_page + '&o={}'.format(num * 100 + 1)
        headlines_num = extract_headlines(url=current_page_url, session=session, news_list=news_list)
        num += 1
        news_quantity += headlines_num
        if(headlines_num == 0):
            break
        print('Loaded headlines: ', len(news_list), flush=True)
        
    return pd.DataFrame(news_list)

In [12]:
def load_news_by_tegs_in_csv_from_marketwatch(tegs_list, session):
    for teg in tegs_list:
        teg_url = 'https://www.marketwatch.com/search?q={}&m=Keyword&rpp=100&mp=159&bd=false&rs=true'.format(teg)
        print('----Loading for', teg)
        teg_pandas_dataframe = load_news(url_first_page=teg_url, session=session)
        
        if not os.path.exists("marketwatch"):
            os.makedirs("marketwatch")
        teg_pandas_dataframe.to_csv('marketwatch/{}.csv'.format(teg))
        
    print('-------Loading finished-------')

In [15]:
tegs = ['amazon', 'tesla', 'Boeing', 'Facebook', 'NVIDIA', 'Netflix', 'AMD', 'Alphabet', 
              'Disney', 'VISA', 'INTEL', 'Mastercard']
load_news_by_tegs_in_csv_from_marketwatch(tegs_list=tegs, session=s)

----Loading for amazon
Loaded headlines:  99
Loaded headlines:  199
Loaded headlines:  299
Loaded headlines:  399
Loaded headlines:  499
Loaded headlines:  599
Loaded headlines:  699
Loaded headlines:  799
Loaded headlines:  899
Loaded headlines:  999
----Loading for tesla
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  201
----Loading for Boeing
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  300
Loaded headlines:  400
Loaded headlines:  500
Loaded headlines:  576
----Loading for Facebook
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  300
Loaded headlines:  400
Loaded headlines:  500
Loaded headlines:  600
Loaded headlines:  604
----Loading for NVIDIA
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  229
----Loading for Netflix
Loaded headlines:  100
Loaded headlines:  170
----Loading for AMD
Loaded headlines:  100
Loaded headlines:  138
----Loading for Alphabet
Loaded headlines:  100
Loaded headlines:  200
Loaded head